In [1]:
#Importando Bibliotecas Pandas e Numpy
import pandas as pd
import numpy as np
import os, sys

#Importando BiBliotecas DASK  (alternativa ao Pandas)
import dask.dataframe as dd

#Otimizador Giovanni
from sklearnex import patch_sklearn 
from sklearn.preprocessing import LabelEncoder

patch_sklearn()
le = LabelEncoder()

CSV_PATH = "/home/gi/Desktop/Semestre8/Inteligencia_Computacional/TP1_csvs" # Path Giovanni
# CSV_PATH = "E:/Documentos/CEFET/OneDrive/Documentos/2ECOM067_INTELIGENCIA-COMPUTACIONAL-I_T01/TP1/bases/" # Path Jorge

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [2]:
import warnings
import re
warnings.simplefilter("ignore")

In [3]:
# Renomeando colunas evitando duplicatas
def renomear_seguro(cols):
    novas = []
    vistos = set()
    for col in cols:
        base = re.sub(r'[^\w]', '', col)
        nome = base
        i = 1
        while nome in vistos:
            nome = f"{base}_{i}"
            i += 1
        vistos.add(nome)
        novas.append(nome)
    return novas

In [4]:
#Carregando Base de dados principal
df_test  = pd.read_csv(f"{CSV_PATH}/application_test.csv")

In [5]:
# Como o volume de informações é muito grande, iremos ao invés de montar uma base única entre a base principal e as 
# demais tabelas auxiliares, iremos criar 4 bases (conforme descrito abaixo), realizar as devidas simulações e utilizar 
# as estratégias de emsemble VotingClassifier e StackingClassifier.
# Bases que serão construídas:
#   -   df_01 = application_train + bureau + bureau_balance
#   -   df_02 = application_train + previous_application + POS_CASH_balance
#   -   df_03 = application_train + previous_application + installments_payments
#   -   df_04 = application_train + previous_application + credit_card_balance

## DataFrame 01 = application_train + bureau + bureau_balance

In [6]:
#Carregamento de Bases de Dados Auxiliares
df_auxiliar_bureau                          = pd.read_csv(f"{CSV_PATH}/bureau.csv")
df_auxiliar_bureau_balance                  = pd.read_csv(f"{CSV_PATH}/bureau_balance.csv")

In [7]:
#Copia do df_test
df_01 = df_test.copy()

In [8]:
#Realização do left join de df_test com df_auxiliar_bureau
# DF da esquerda = df_test
# DF da direita  = df_auxiliar_bureau
df_01 = df_01.join(df_auxiliar_bureau, on='SK_ID_CURR', lsuffix = ' ', how='left')
df_01 = df_01.drop(columns={"SK_ID_CURR"}) #Excluir coluna duplicada

#Realização do left join de df_01 com df_auxiliar_bureau_balance
# DF da esquerda = df_01
# DF da direita  = df_auxiliar_bureau_balance
df_01 = df_01.join(df_auxiliar_bureau_balance, on='SK_ID_BUREAU', lsuffix = ' ',  how='left')
df_01 = df_01.drop(columns={"SK_ID_BUREAU"}) #Excluir coluna duplicada

# Limpa caracteres especiais das colunas
df_01.columns = renomear_seguro(df_01.columns)

# Label encoder nas colunas categóricas

colunas_categoricas = df_01.select_dtypes(include=['object', 'category']).columns.tolist()
for coluna in colunas_categoricas:
    df_01[coluna] = le.fit_transform(df_01[coluna].astype(str))
df_01.to_csv(f"{CSV_PATH}/df_01_test.csv", index=False) #Salvar df_01 em CSV

## DataFrame 02 = application_train + previous_application + POS_CASH_balance

In [9]:
#Carregamento de Bases de Dados Auxiliares
df_auxiliar_previous_application            = pd.read_csv(f"{CSV_PATH}/previous_application.csv")
df_auxiliar_POS_CASH_balance                = pd.read_csv(f"{CSV_PATH}/POS_CASH_balance.csv")

In [10]:
#Copia do df_test
df_02 = df_test.copy()

In [11]:
#Realização do left join de df_test com df_auxiliar_previous_application
# DF da esquerda = df_test
# DF da direita  = df_auxiliar_previous_application
df_02 = df_02.join(df_auxiliar_previous_application, on='SK_ID_CURR', lsuffix = ' ', how='left')
df_02 = df_02.drop(columns={"SK_ID_CURR"}) #Excluir coluna duplicada

#Realização do left join de df_02 com df_auxiliar_POS_CASH_balance
# DF da esquerda = df_02
# DF da direita  = df_auxiliar_POS_CASH_balance
df_02 = df_02.join(df_auxiliar_POS_CASH_balance, on='SK_ID_PREV', lsuffix = ' ',  how='left')
df_02 = df_02.drop(columns={"SK_ID_PREV"}) #Excluir coluna duplicada

# Limpa caracteres especiais das colunas
df_02.columns = renomear_seguro(df_02.columns)
colunas_categoricas = df_02.select_dtypes(include=['object', 'category']).columns.tolist()
for coluna in colunas_categoricas:
    try:
        df_02[coluna] = le.fit_transform(df_02[coluna].astype(str))
    except ValueError:
        print(f"Erro ao converter coluna {coluna} para string. Verifique os dados.")
        raise ValueError
df_02.to_csv(f"{CSV_PATH}/df_02_test.csv", index=False) #Salvar df_02 em CSV

## DataFrame 03 = application_train + previous_application + installments_payments

In [12]:
#Carregamento de Bases de Dados Auxiliares
#df_auxiliar_previous_application            = pd.read_csv(f"{CSV_PATH}/previous_application.csv")
df_auxiliar_installments_payments           = pd.read_csv(f"{CSV_PATH}/installments_payments.csv")

In [13]:
#Copia do df_test
df_03 = df_test.copy()

In [14]:
#Realização do left join de df_test com df_auxiliar_previous_application
# DF da esquerda = df_test
# DF da direita  = df_auxiliar_previous_application
df_03 = df_03.join(df_auxiliar_previous_application, on='SK_ID_CURR', lsuffix = ' ', how='left')
df_03 = df_03.drop(columns={"SK_ID_CURR"}) #Excluir coluna duplicada

#Realização do left join de df_03 com df_auxiliar_installments_payments
# DF da esquerda = df_03
# DF da direita  = df_auxiliar_installments_payments
df_03 = df_03.join(df_auxiliar_installments_payments, on='SK_ID_PREV', lsuffix = ' ',  how='left')
df_03 = df_03.drop(columns={"SK_ID_PREV"}) #Excluir coluna duplicada

# Limpa caracteres especiais das colunas
df_03.columns = renomear_seguro(df_03.columns)
colunas_categoricas = df_03.select_dtypes(include=['object', 'category']).columns.tolist()
for coluna in colunas_categoricas:
    df_03[coluna] = le.fit_transform(df_03[coluna].astype(str))
df_03.to_csv(f"{CSV_PATH}/df_03_test.csv", index=False) #Salvar df_03 em CSV

## DataFrame 04 = application_train + previous_application + credit_card_balance

In [15]:
#Carregamento de Bases de Dados Auxiliares
#df_auxiliar_previous_application            = pd.read_csv(f"{CSV_PATH}/previous_application.csv")
df_auxiliar_credit_card_balances            = pd.read_csv(f"{CSV_PATH}/credit_card_balance.csv")

In [16]:
#Copia do df_test
df_04 = df_test.copy()

In [17]:
#Realização do left join de df_test com df_auxiliar_previous_application
# DF da esquerda = df_test
# DF da direita  = df_auxiliar_previous_application
df_04 = df_04.join(df_auxiliar_previous_application, on='SK_ID_CURR', lsuffix = ' ', how='left')
df_04 = df_04.drop(columns={"SK_ID_CURR"}) #Excluir coluna duplicada

#Realização do left join de df_03 com df_auxiliar_credit_card_balances
# DF da esquerda = df_04
# DF da direita  = df_auxiliar_credit_card_balances
df_04 = df_04.join(df_auxiliar_credit_card_balances, on='SK_ID_PREV', lsuffix = ' ',  how='left')
df_04 = df_04.drop(columns={"SK_ID_PREV"}) #Excluir coluna duplicada

# Limpa caracteres especiais das colunas
df_04.columns = renomear_seguro(df_04.columns)

colunas_categoricas = df_04.select_dtypes(include=['object', 'category']).columns.tolist()
for coluna in colunas_categoricas:
    df_04[coluna] = le.fit_transform(df_04[coluna].astype(str))
df_04.to_csv(f"{CSV_PATH}/df_04_test.csv", index=False) #Salvar df_04 em CSV 

In [18]:
# Tamanhos dos Data Frames criados
print(f"df_test shape {df_test.shape}, df_01 shape {df_01.shape}, df_02 shape {df_02.shape}, df_03 shape {df_03.shape}, df_04 shape {df_04.shape}")

df_test shape (48744, 121), df_01 shape (48744, 139), df_02 shape (48744, 164), df_03 shape (48744, 164), df_04 shape (48744, 179)
